# Message模块

## 简介

`Message`是对大语言模型输入输出信息的封装，可以简化大家和大语言模型的交互。

在`ERNIE Bot Agent`中，主要有如下4类`Message`：

* `HumanMessage`：用户输入给模型的普通信息，比如聊天的问题。
* `SystemMessage`：用户输入给模型的全局信息，比如角色扮演的指令、输出格式设置的指令。
* `AIMessage`：模型返回的信息，比如聊天的回答、触发`Function call`的回答。
* `FunctionMessage`：上一轮模型的输出是带有`Funciton call`的`AIMessage`，则用户需要首先调用`Function`，然后将`Function`的结果输入给大语言模型。

`Message`模块的详细API接口，请参考[文档](https://ernie-bot-agent.readthedocs.io/zh-cn/latest/package/erniebot_agent/message/)。

## 上手使用Message

### 安装

大家可以参考[官方文档](https://github.com/PaddlePaddle/ERNIE-Bot-SDK)，进行安装ERNIE Bot SDK。


### 鉴权

大家在使用ERNIE Bot Agent之前，需要完成鉴权步骤：

* 在[AI Studio星河社区](https://aistudio.baidu.com/index)注册并登录账号
* 在个人中心的[访问令牌页面](https://aistudio.baidu.com/index/accessToken)获取用户凭证`Access Token`
* 通过环境变量或者`Python`代码设置`Access Token`

In [13]:
# %env EB_AGENT_ACCESS_TOKEN=xxxxxxx

import os

os.environ["EB_AGENT_ACCESS_TOKEN"] = "xxxxxxx"

## 使用Message

首先，导入必要的依赖库。

In [14]:
import os
import json
import asyncio
import erniebot
from erniebot_agent.chat_models import ERNIEBot
from erniebot_agent.memory import HumanMessage, AIMessage, SystemMessage, FunctionMessage

大家在使用`ERNIE Bot SDK`调用文心一言进行多轮对话时，需要按照规范定义每轮对话的信息（如下），稍显复杂。

In [ ]:
erniebot.api_type = "aistudio"
erniebot.access_token = "<access-token-for-aistudio>"
messages = [
    {"role": "user", "content": "我在深圳，周末可以去哪里玩"},
    {"role": "assistant", "content": "深圳有许多著名的景点，以下是三个推荐景点：1. 深圳世界之窗，2. 深圳欢乐谷，3. 深圳东部华侨城。"},
    {"role": "user", "content": "从你推荐的三个景点中，选出最值得去的景点是什么，直接给出景点名字即可"},
]
response = erniebot.ChatCompletion.create(model="ernie-3.5", messages=messages)

print(response.get_result())

如果基于`ERNIE Bot Agent`调用文心一言，大家使用`Message`模块，可以较好地简化代码。

In [15]:
async def demo():
    model = ERNIEBot(model="ernie-3.5")
    # 使用Message模块
    messages = [
        HumanMessage("我在深圳，周末可以去哪里玩"),
        AIMessage("深圳有许多著名的景点，以下是三个推荐景点：1. 深圳世界之窗，2. 深圳欢乐谷，3. 深圳东部华侨城。"),
        HumanMessage("从你推荐的三个景点中，选出最值得去的景点是什么，直接给出景点名字即可"),
    ]
    ai_message = await model.chat(messages=messages)
    print(ai_message.content)


await demo()

最值得去的景点是深圳欢乐谷。


创建各种`Message`的示例代码如下：

In [16]:
human_message = HumanMessage(content="你好，你是谁？")

system_message = SystemMessage(content="你是一名数学老师，使用浅显易懂的方法来回答问题")

result = {"temperature": 25, "unit": "摄氏度"}
function_message = FunctionMessage(
    name="get_current_temperature", content=json.dumps(result, ensure_ascii=False)
)

print(human_message)
print(system_message)
print(function_message)

<role: 'user', content: '你好，你是谁？'>
<role: 'system', content: '你是一名数学老师，使用浅显易懂的方法来回答问题', token_count: 23>
<role: 'function', name: 'get_current_temperature', content: '{"temperature": 25, "unit": "摄氏度"}'>


使用`SystemMessage`的示例代码如下：

In [17]:
async def demo():
    model = ERNIEBot(model="ernie-3.5")
    system_message = SystemMessage(content="你是一名数学老师，尽量使用浅显易懂的方法来解答问题")
    messages = [HumanMessage("勾股定理是什么")]
    ai_message = await model.chat(messages=messages, system=system_message.content)
    print(ai_message.content)


await demo()

勾股定理是一个基本的几何定理，它指出在一个直角三角形中，直角边的平方和等于斜边的平方。具体来说，如果直角三角形的两个直角边长度分别为a和b，斜边长度为c，那么勾股定理可以表示为：a² + b² = c²。

这个定理在中国古代被称为勾股定理或商高定理，因为在周朝时期，商高提出了“勾三股四弦五”的特例。在西方，这个定理最早是由古希腊的毕达哥拉斯学派提出的，他们用演绎法证明了这一定理。

勾股定理是人类早期发现并证明的重要数学定理之一，也是用代数思想解决几何问题的最重要的工具之一。它有多种证明方法，是数学定理中证明方法最多的定理之一。同时，勾股定理也是数形结合的纽带之一。

总的来说，勾股定理是一个非常重要的几何定理，它在数学和科学领域有广泛的应用。
